# 🎓 Week 18 - Day 2: Dockerfiles for ML Projects

## Today's Goals:
✅ Master advanced Dockerfile instructions

✅ Understand layer caching strategies

✅ Learn multi-stage builds

✅ Write production-ready Dockerfiles

✅ Apply best practices for ML deployments

---

## ⚠️ IMPORTANT: How to Use This Notebook

This notebook contains:
- 📚 **Explanations and theory** (read here)
- 🖥️ **Docker commands** (copy and run in your **Terminal/CMD**)
- 📝 **File contents** (create these files in a text editor)
- ✅ **Expected outputs** (what you should see)

### 🔴 Commands with 🖥️ emoji → Run in Terminal/CMD, NOT in Jupyter!

### 🔵 Commands with 🐍 emoji → Run in Jupyter cells (these are Python code)

---

## 📦 Prerequisites Checklist

Before starting, make sure you have:

- ✅ Docker Desktop installed and running
- ✅ Completed Day 1 (basic Docker commands)
- ✅ Text editor ready (VS Code, Notepad++, or any editor)
- ✅ Terminal/CMD window open

**Test Docker is working:**

```bash
# 🖥️ Run in Terminal:
docker --version
docker images
```

**Expected Output:**
```
Docker version 24.0.6, build ed223bc
REPOSITORY   TAG       IMAGE ID       CREATED       SIZE
```

✅ If you see this, you're ready to go!

---

## 📝 Part 1: Dockerfile Instructions Deep Dive

### 🎯 Objective
Understand every Dockerfile instruction and when to use it.

### 📚 Key Instructions

| Instruction | Purpose | Example |
|-------------|---------|--------|
| **FROM** | Base image | `FROM python:3.11-slim` |
| **WORKDIR** | Set working directory | `WORKDIR /app` |
| **COPY** | Copy files to container | `COPY app.py .` |
| **RUN** | Execute commands during BUILD | `RUN pip install flask` |
| **ENV** | Set environment variables | `ENV PORT=8000` |
| **EXPOSE** | Document port | `EXPOSE 8000` |
| **CMD** | Default command at runtime | `CMD ["python", "app.py"]` |
| **ENTRYPOINT** | Main executable | `ENTRYPOINT ["python"]` |

### 💡 Key Difference
- **RUN** = Executes during `docker build` (installing packages)
- **CMD** = Executes when `docker run` starts the container

### 🛠️ Exercise 1: Create a Basic Dockerfile

**Step 1: Create project directory**

```bash
# 🖥️ Run in Terminal:
mkdir docker-ml-project
cd docker-ml-project
```

**Step 2: Create `app.py`**

Create a file called `app.py` in your project directory with:

```python
from fastapi import FastAPI
import numpy as np

app = FastAPI()

@app.get("/")
def home():
    return {"message": "ML API is running!"}

@app.get("/predict")
def predict():
    # Simple prediction simulation
    prediction = np.random.randint(0, 100)
    return {"prediction": int(prediction)}
```

**Step 3: Create `requirements.txt`**

```
fastapi==0.104.1
uvicorn==0.24.0
numpy==1.24.3
```

**Step 4: Create `Dockerfile`**

Create a file called `Dockerfile` (no extension) with:

```dockerfile
# Use Python 3.11 slim image
FROM python:3.11-slim

# Set working directory
WORKDIR /app

# Copy requirements first (for caching)
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY app.py .

# Expose port
EXPOSE 8000

# Run the application
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

**Step 5: Build the image**

```bash
# 🖥️ Run in Terminal:
docker build -t ml-api:v1 .
```

**Expected Output:**
```
[+] Building 45.2s (10/10) FINISHED
 => [internal] load build definition from Dockerfile
 => => transferring dockerfile: 350B
 => [1/5] FROM docker.io/library/python:3.11-slim
 => [2/5] WORKDIR /app
 => [3/5] COPY requirements.txt .
 => [4/5] RUN pip install --no-cache-dir -r requirements.txt
 => [5/5] COPY app.py .
 => exporting to image
```

**Step 6: Run the container**

```bash
# 🖥️ Run in Terminal:
docker run -d -p 8000:8000 --name ml-container ml-api:v1
```

**Step 7: Test it!**

Open your browser to: http://localhost:8000

You should see:
```json
{"message": "ML API is running!"}
```

✅ **Success!** You've created your first ML Dockerfile!

---

## 🧱 Part 2: Layer Caching Strategy

### 🎯 Objective
Understand Docker layer caching to speed up builds from 5 minutes to 30 seconds!

### 📚 How Docker Caching Works

Docker builds images in **layers**. Each instruction creates a layer:

```dockerfile
FROM python:3.11-slim      # Layer 1
WORKDIR /app               # Layer 2
COPY requirements.txt .    # Layer 3
RUN pip install ...        # Layer 4 (SLOW - installs packages)
COPY app.py .              # Layer 5
```

**Key Insight:** If a layer hasn't changed, Docker reuses it from cache!

### ❌ BAD Dockerfile (No Caching Benefits)

```dockerfile
FROM python:3.11-slim
WORKDIR /app

# Copies EVERYTHING first
COPY . .

# Now installs dependencies
RUN pip install -r requirements.txt

CMD ["python", "app.py"]
```

**Problem:** Change `app.py` → Layer 3 changes → Layer 4 rebuilds → pip reinstalls everything! (5 minutes)

### ✅ GOOD Dockerfile (Smart Caching)

```dockerfile
FROM python:3.11-slim
WORKDIR /app

# Copy ONLY requirements first
COPY requirements.txt .

# Install dependencies (cached unless requirements.txt changes)
RUN pip install -r requirements.txt

# Copy application code LAST
COPY . .

CMD ["python", "app.py"]
```

**Benefit:** Change `app.py` → Only Layer 5 rebuilds → pip uses cache! (30 seconds)

### 💡 Golden Rule

**Order matters:**
1. Things that **rarely change** go FIRST (base image, system packages)
2. Things that **sometimes change** go MIDDLE (dependencies)
3. Things that **often change** go LAST (your application code)

### 🛠️ Exercise 2: Test Layer Caching

**Step 1: Build the image (first time)**

```bash
# 🖥️ Run in Terminal:
docker build -t ml-api:v2 .
```

**Note the time:** Maybe 30-60 seconds

**Step 2: Rebuild immediately (no changes)**

```bash
# 🖥️ Run in Terminal:
docker build -t ml-api:v2 .
```

**Expected Output:**
```
[+] Building 0.5s (10/10) FINISHED
 => CACHED [1/5] FROM python:3.11-slim
 => CACHED [2/5] WORKDIR /app
 => CACHED [3/5] COPY requirements.txt .
 => CACHED [4/5] RUN pip install ...
 => CACHED [5/5] COPY app.py .
```

**Notice:** All layers are **CACHED**. Build takes 0.5 seconds!

**Step 3: Modify app.py**

Change the message in `app.py`:
```python
return {"message": "ML API v2 is running!"}
```

**Step 4: Rebuild**

```bash
# 🖥️ Run in Terminal:
docker build -t ml-api:v2 .
```

**Expected Output:**
```
 => CACHED [1/5] FROM python:3.11-slim
 => CACHED [2/5] WORKDIR /app
 => CACHED [3/5] COPY requirements.txt .
 => CACHED [4/5] RUN pip install ...    ← Still cached!
 => [5/5] COPY app.py .                 ← Only this rebuilds!
```

✅ **See the power of caching?** Only the changed layer rebuilds!

---

## 🏗️ Part 3: Multi-Stage Builds

### 🎯 Objective
Create small production images by using multiple stages.

### 📚 What Are Multi-Stage Builds?

Use **multiple FROM statements** in one Dockerfile:
- **Stage 1 (Builder):** Has everything needed to build/train (2GB)
- **Stage 2 (Production):** Has only what's needed to run (200MB)

### 💡 Use Cases
- Training ML models (need training libs)
- Compiling code (need compilers)
- Building assets (need build tools)

**Final image:** Only contains runtime dependencies + trained model!

### 🛠️ Exercise 3: Multi-Stage Build

**Scenario:** Train a model in Stage 1, run inference in Stage 2

**Step 1: Create `train.py`**

```python
# Simple training simulation
import pickle
import numpy as np

print("Training model...")

# Simulate training
model = {"weights": np.random.rand(10, 10), "version": "v1.0"}

# Save model
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model trained and saved to model.pkl")
```

**Step 2: Create `inference.py`**

```python
from fastapi import FastAPI
import pickle

app = FastAPI()

# Load model
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

@app.get("/")
def home():
    return {"status": "Model loaded", "version": model["version"]}

@app.get("/predict")
def predict():
    # Use model for prediction
    result = model["weights"].sum()
    return {"prediction": float(result)}
```

**Step 3: Create multi-stage Dockerfile**

```dockerfile
# ============================================
# STAGE 1: Builder (Training)
# ============================================
FROM python:3.11 AS builder

WORKDIR /build

# Install training dependencies
COPY requirements.txt .
RUN pip install --user numpy scikit-learn

# Copy and run training script
COPY train.py .
RUN python train.py

# ============================================
# STAGE 2: Production (Inference Only)
# ============================================
FROM python:3.11-slim

WORKDIR /app

# Copy ONLY runtime dependencies from builder
COPY --from=builder /root/.local /root/.local

# Copy ONLY the trained model from builder
COPY --from=builder /build/model.pkl .

# Install production dependencies (lightweight)
RUN pip install fastapi uvicorn

# Copy inference code
COPY inference.py .

EXPOSE 8000

CMD ["uvicorn", "inference:app", "--host", "0.0.0.0", "--port", "8000"]
```

**Step 4: Build the image**

```bash
# 🖥️ Run in Terminal:
docker build -t ml-multistage:v1 .
```

**Expected Output:**
```
[+] Building 68.3s (15/15) FINISHED
 => [builder 1/5] FROM python:3.11
 => [builder 2/5] WORKDIR /build
 => [builder 3/5] COPY requirements.txt .
 => [builder 4/5] RUN pip install ...
 => [builder 5/5] RUN python train.py         ← Training happens here!
 => [stage-1 1/5] FROM python:3.11-slim
 => [stage-1 2/5] COPY --from=builder ...     ← Copying from builder
 => [stage-1 3/5] COPY --from=builder /build/model.pkl .
```

**Step 5: Check image size**

```bash
# 🖥️ Run in Terminal:
docker images ml-multistage:v1
```

**Expected Output:**
```
REPOSITORY        TAG       SIZE
ml-multistage     v1        220MB   ← Small!
```

Compare to a single-stage build: ~1.5GB

✅ **Result:** 85% smaller image!

---

## 🔧 Part 4: Environment Variables (ENV & ARG)

### 🎯 Objective
Learn when to use ENV (runtime) vs ARG (build-time).

### 📚 ENV vs ARG

| Feature | ENV | ARG |
|---------|-----|-----|
| **When** | Runtime | Build time |
| **Override** | `docker run -e VAR=value` | `docker build --build-arg VAR=value` |
| **In Code** | `os.getenv('VAR')` | Not accessible at runtime |
| **Use For** | API keys, model paths | Python version, base image |

### 💡 Examples

**ENV (Runtime variables):**
```dockerfile
ENV MODEL_PATH=/models/best.pkl
ENV API_KEY=default_key
ENV LOG_LEVEL=INFO
```

**ARG (Build-time variables):**
```dockerfile
ARG PYTHON_VERSION=3.11
FROM python:${PYTHON_VERSION}-slim
```

### 🛠️ Exercise 4: Using ENV Variables

**Step 1: Create `config_app.py`**

```python
from fastapi import FastAPI
import os

app = FastAPI()

# Read environment variables
MODEL_PATH = os.getenv("MODEL_PATH", "default_model.pkl")
LOG_LEVEL = os.getenv("LOG_LEVEL", "INFO")
API_VERSION = os.getenv("API_VERSION", "v1.0")

@app.get("/")
def home():
    return {
        "model_path": MODEL_PATH,
        "log_level": LOG_LEVEL,
        "api_version": API_VERSION
    }
```

**Step 2: Create Dockerfile with ENV**

```dockerfile
FROM python:3.11-slim

WORKDIR /app

# Set environment variables
ENV MODEL_PATH=/models/production.pkl
ENV LOG_LEVEL=INFO
ENV API_VERSION=v2.0

RUN pip install fastapi uvicorn

COPY config_app.py .

EXPOSE 8000

CMD ["uvicorn", "config_app:app", "--host", "0.0.0.0", "--port", "8000"]
```

**Step 3: Build and run**

```bash
# 🖥️ Run in Terminal:
docker build -t config-api .
docker run -d -p 8000:8000 --name config-container config-api
```

**Step 4: Test default ENV values**

Visit: http://localhost:8000

```json
{
  "model_path": "/models/production.pkl",
  "log_level": "INFO",
  "api_version": "v2.0"
}
```

**Step 5: Override ENV at runtime**

```bash
# 🖥️ Run in Terminal:
docker stop config-container
docker rm config-container

# Run with custom ENV values
docker run -d -p 8000:8000 \
  -e MODEL_PATH=/models/custom.pkl \
  -e LOG_LEVEL=DEBUG \
  -e API_VERSION=v3.0 \
  --name config-container config-api
```

Visit: http://localhost:8000

```json
{
  "model_path": "/models/custom.pkl",
  "log_level": "DEBUG",
  "api_version": "v3.0"
}
```

✅ **ENV variables overridden at runtime!**

---

## 🚫 Part 5: .dockerignore File

### 🎯 Objective
Exclude unnecessary files from Docker builds to save time and space.

### 📚 What is .dockerignore?

Like `.gitignore` but for Docker. Tells Docker which files to EXCLUDE when copying.

### ⚠️ Without .dockerignore:
- 5GB dataset copied into image
- .git history (100MB+) included
- Virtual environments copied
- Build takes 10+ minutes

### ✅ With .dockerignore:
- Only essential code copied
- Image size: 200MB instead of 5GB
- Build time: 30 seconds instead of 10 minutes

### 🛠️ Exercise 5: Create .dockerignore

**Step 1: Create `.dockerignore` file**

In your project directory, create a file called `.dockerignore` (note the dot):

```
# Python
__pycache__/
*.pyc
*.pyo
*.pyd
.pytest_cache/
.coverage
htmlcov/

# Virtual environments
venv/
.venv/
env/
ENV/

# Git
.git/
.gitignore
.gitattributes

# IDE
.vscode/
.idea/
*.swp
*.swo
.DS_Store

# Data files (DO NOT copy large datasets!)
data/raw/
data/processed/
*.csv
*.xlsx
*.parquet
*.h5

# Models (train them inside container or mount as volume)
models/*.pkl
models/*.h5
models/*.pt

# Documentation
docs/
*.md
LICENSE

# Docker
Dockerfile*
docker-compose*.yml
.dockerignore

# CI/CD
.github/
.gitlab-ci.yml
```

**Step 2: Test it**

```bash
# 🖥️ Run in Terminal:
# Create some files that should be ignored
mkdir data
echo "test" > data/test.csv
echo "test" > README.md
mkdir .vscode

# Build image
docker build -t dockerignore-test .

# Check what's inside
docker run --rm dockerignore-test ls -la
```

**Expected:** You should NOT see `data/`, `README.md`, or `.vscode/`

✅ **Files successfully excluded!**

---

## ⭐ Part 6: Dockerfile Best Practices

### 🎯 10 Production Best Practices

#### 1️⃣ **Use Specific Tags**
```dockerfile
# ❌ Bad
FROM python:latest

# ✅ Good
FROM python:3.11-slim
```
**Why:** `latest` changes over time, breaking reproducibility

#### 2️⃣ **Minimize Layers**
```dockerfile
# ❌ Bad (3 layers)
RUN apt-get update
RUN apt-get install -y curl
RUN rm -rf /var/lib/apt/lists/*

# ✅ Good (1 layer)
RUN apt-get update && \
    apt-get install -y curl && \
    rm -rf /var/lib/apt/lists/*
```
**Why:** Fewer layers = smaller image

#### 3️⃣ **Leverage Cache (Requirements First)**
```dockerfile
# ✅ Good order
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
```

#### 4️⃣ **Use .dockerignore**
Always create a `.dockerignore` file

#### 5️⃣ **Multi-Stage for Production**
Use builder stages to keep final image small

#### 6️⃣ **Don't Run as Root**
```dockerfile
# Create non-root user
RUN useradd -m -u 1000 appuser
USER appuser
```
**Why:** Security best practice

#### 7️⃣ **Pin Dependencies**
```
# requirements.txt
fastapi==0.104.1
numpy==1.24.3
```
**Why:** Reproducible builds

#### 8️⃣ **Add Health Checks**
```dockerfile
HEALTHCHECK --interval=30s --timeout=3s \
  CMD curl -f http://localhost:8000/health || exit 1
```

#### 9️⃣ **Label Your Images**
```dockerfile
LABEL maintainer="your-email@example.com"
LABEL version="1.0"
LABEL description="ML API for predictions"
```

#### 🔟 **Clean Up in Same Layer**
```dockerfile
RUN apt-get update && \
    apt-get install -y build-essential && \
    # ... do stuff ... && \
    apt-get remove -y build-essential && \
    rm -rf /var/lib/apt/lists/*
```
**Why:** Reduces image size

---

## 🤖 Part 7: Complete Production Dockerfile

### 🎯 Objective
Create a production-ready Dockerfile with ALL best practices.

### 📝 Production Dockerfile Template

```dockerfile
# ==============================================
# Production ML API Dockerfile
# ==============================================

FROM python:3.11-slim

# Metadata
LABEL maintainer="your-email@example.com"
LABEL version="1.0.0"
LABEL description="Production ML API with FastAPI"

# Install system dependencies
RUN apt-get update && apt-get install -y \
    build-essential \
    curl \
    && rm -rf /var/lib/apt/lists/*

# Create non-root user
RUN useradd -m -u 1000 appuser && \
    mkdir -p /app && \
    chown -R appuser:appuser /app

# Set working directory
WORKDIR /app

# Copy requirements and install (as root for permissions)
COPY requirements.txt .
RUN pip install --no-cache-dir --upgrade pip && \
    pip install --no-cache-dir -r requirements.txt

# Copy application files
COPY --chown=appuser:appuser . .

# Switch to non-root user
USER appuser

# Environment variables
ENV MODEL_PATH=/app/models/model.pkl
ENV LOG_LEVEL=INFO
ENV WORKERS=4

# Expose port
EXPOSE 8000

# Health check
HEALTHCHECK --interval=30s --timeout=5s --start-period=5s --retries=3 \
  CMD curl -f http://localhost:8000/health || exit 1

# Run application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000", "--workers", "4"]
```

### ✅ This Dockerfile includes:
- ✅ Specific Python version (3.11-slim)
- ✅ Metadata labels
- ✅ System dependencies installed and cleaned up
- ✅ Non-root user (appuser)
- ✅ Requirements copied first for caching
- ✅ No cache during pip install
- ✅ Proper file ownership
- ✅ Environment variables
- ✅ Health check
- ✅ Production-ready CMD

---

## 🎯 Part 8: Beginner Challenge

### 🏆 Challenge: Production-Ready ML API

**Your Task:**

Create a complete ML API with a production-ready Dockerfile that:

**Requirements:**

1. **Create `ml_api.py`** with:
   - FastAPI app
   - `GET /` endpoint (health check)
   - `POST /predict` endpoint (accepts `{"value": number}`, returns prediction)
   - `GET /model-info` endpoint (returns model metadata)
   - Load a simple model (pickle file)

2. **Create `requirements.txt`** with pinned versions:
   - fastapi
   - uvicorn
   - numpy
   - scikit-learn

3. **Create production Dockerfile** that:
   - Uses multi-stage build (optional: train model in stage 1)
   - Leverages layer caching
   - Runs as non-root user
   - Includes health check
   - Has proper labels
   - Uses .dockerignore

4. **Build and run:**
   ```bash
   docker build -t ml-challenge:v1 .
   docker run -d -p 8000:8000 --name ml-api ml-challenge:v1
   ```

5. **Test:**
   - Visit: http://localhost:8000
   - Test predict: `curl -X POST http://localhost:8000/predict -H "Content-Type: application/json" -d '{"value": 42}'`

**Expected Output:**
```json
{
  "prediction": 84,
  "model_version": "v1.0",
  "timestamp": "2024-12-03T10:30:00"
}
```

**Bonus Points:**
- ⭐ Image size under 300MB
- ⭐ Build time under 2 minutes
- ⭐ Proper error handling in API
- ⭐ Environment variables for configuration
- ⭐ Docker image scan passes (no critical vulnerabilities)

### 💡 Hints

<details>
<summary>Click for hints</summary>

**Hint 1:** Start with requirements.txt first, then write code

**Hint 2:** For multi-stage:
```dockerfile
FROM python:3.11 AS builder
# Train model here
RUN python train_model.py

FROM python:3.11-slim
COPY --from=builder /app/model.pkl .
```

**Hint 3:** Health check endpoint:
```python
@app.get("/health")
def health():
    return {"status": "healthy"}
```

**Hint 4:** Non-root user:
```dockerfile
RUN useradd -m -u 1000 appuser
USER appuser
```

</details>

---

## 📚 Summary

### 🎯 What We Learned:

**1. Dockerfile Instructions:**
- FROM, WORKDIR, COPY, RUN, ENV, ARG, EXPOSE, CMD, ENTRYPOINT
- When to use each instruction
- Difference between RUN (build) and CMD (runtime)

**2. Layer Caching:**
- How Docker caching works (layer by layer)
- Why order matters (rarely changing first, frequently changing last)
- 5 min build → 30 sec rebuild with smart caching

**3. Multi-Stage Builds:**
- Building in one stage, running in another
- Reducing image size from 2GB → 200MB
- Perfect for training models or compiling code

**4. Environment Variables:**
- ENV for runtime configuration (API keys, paths)
- ARG for build-time configuration (base image, versions)
- Overriding ENV at runtime with -e flag

**5. .dockerignore:**
- Excluding unnecessary files (data, .git, venv)
- Reducing build time and image size
- Essential for ML projects with large datasets

**6. Best Practices:**
- 10 production best practices
- Security (non-root user)
- Maintainability (labels, health checks)
- Efficiency (minimize layers, leverage cache)

**7. Production Dockerfile:**
- Complete template with all best practices
- Ready for real ML deployments
- Optimized for size, speed, and security

### 💡 Key Takeaways:

✅ **Order matters** - Dependencies first, code last

✅ **Layer caching** is your friend - Use it wisely

✅ **Small images** deploy faster - Multi-stage builds help

✅ **Security** matters - Don't run as root

✅ **.dockerignore** saves time - Always use it

✅ **Best practices** aren't optional - They prevent production issues

### 🚀 What's Next:

**Tomorrow (Day 3): CI/CD with GitHub Actions**
- Automate Docker builds
- Push images to registries
- Deploy automatically
- Testing in pipelines

### 📝 Today's Project:

**Create a production-ready Dockerfile for your Week 17 FastAPI app!**

Apply everything you learned:
- ✅ Smart layer caching
- ✅ Multi-stage build (if needed)
- ✅ Environment variables
- ✅ .dockerignore file
- ✅ All 10 best practices
- ✅ Non-root user
- ✅ Health check

**Great job! You're now a Dockerfile pro! 🎉**

---

## 🔧 Troubleshooting

### Common Issues and Solutions:

#### Issue 1: "Cannot connect to Docker daemon"
**Solution:**
- Make sure Docker Desktop is running
- Check system tray (Windows) or menu bar (Mac) for Docker icon
- Restart Docker Desktop if needed

#### Issue 2: "Build fails with permission denied"
**Solution:**
- On Linux: `sudo usermod -aG docker $USER`
- Log out and log back in
- Or run with sudo: `sudo docker build ...`

#### Issue 3: "Port already in use"
**Solution:**
```bash
# Find and stop container using port 8000
docker ps
docker stop <container_id>
# Or use different port
docker run -p 8001:8000 ...
```

#### Issue 4: "Image build is very slow"
**Solution:**
- Check your .dockerignore includes large files
- Use a base image closer to what you need
- Optimize layer caching (requirements first)

#### Issue 5: "Container exits immediately"
**Solution:**
```bash
# Check logs
docker logs <container_id>
# Common causes:
# - Wrong CMD/ENTRYPOINT
# - Application crashes
# - Missing dependencies
```

---

## 📖 Additional Resources

**Official Documentation:**
- Docker Best Practices: https://docs.docker.com/develop/dev-best-practices/
- Dockerfile Reference: https://docs.docker.com/engine/reference/builder/

**Cheat Sheets:**
- Docker CLI: https://docs.docker.com/get-started/docker_cheatsheet.pdf
- Dockerfile Instructions: https://kapeli.com/cheat_sheets/Dockerfile.docset/Contents/Resources/Documents/index

**Tools:**
- Dive (explore image layers): https://github.com/wagoodman/dive
- Docker Scan: `docker scan <image_name>`

---

**End of Day 2 Notebook** 🎉